In [ ]:
from lec_utils import *
import lec24_util as util

<div class="alert alert-info" markdown="1">

#### Lecture 24

# Clustering

### EECS 398: Practical Data Science, Winter 2025

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/wn25">github.com/practicaldsc/wn25</a> • 📣 See latest announcements [**here on Ed**](https://edstem.org/us/courses/69737/discussion/5943734) </small>
    
</div>

<script type="text/x-mathjax-config">
 MathJax.Hub.Config({
   TeX: {
     extensions: ["color.js"],
     packages: {"[+]": ["color"]},
   }
 });
 </script>
 <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js?config=TeX-AMS_HTML"></script>

### Please give us feedback! 🙏

- We're looking for your feedback to help improve the course for future offerings and decide where it sits in the overall EECS/DS curriculum.

- If **at least 85% of the class** fills out **both**:
    - This [**internal End-of-Semester Survey**](https://forms.gle/TLomgG6rB3BcLa9s6), **and**
    - the [**Official Campus Evaluations**](https://umich.bluera.com/umich/),<br>
    
    then we will add 1% of extra credit to everyone's overall grade.

- The deadline to fill out both is on **Wednesday, April 23rd at 11:59PM**.

### Agenda 📆

- Today's focus is on **clustering**, an **unsupervised learning** method. We'll focus on $k$-means clustering, the most popular clustering technique, but discuss another clustering technique (agglomerative clustering) as well.

- In our final lecture on Monday, we'll more examples of other techniques in machine learning that are small extensions to what we've covered so far.

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Clustering

---

### The taxonomy of machine learning

<center><img src="imgs/taxonomy.svg" width=600></center>

- In Lectures 11-19, we focused on building models for **regression**.<br><small>In regression, we predict a **continuous** target variable, $y$, using some features, $X$.</small>

- In the past few lectures, we switched our focus to building models for **classification**.<br><small>In classification, we predict a **categorical** target variable, $y$, using some features, $X$.</small>

- Both regression and classification are **supervised learning** methods.<br><small>In both regression and classification, our goal is to predict $y$ from $X$. The datasets we've used already had a $y$ variable.</small>

- What might an **unsupervised learning** problem look like?

### Example: TV show ratings 📺

- Suppose we have the ratings that several customers of a streaming service gave to two popular TV shows: _Modern Family_ and _Stranger Things_.

In [ ]:
util.show_ratings()

- The data naturally falls into three groups, or **clusters**, based on users with similar preferences.<br><small>All we're given are the ratings each customer gave to the two shows; the customers aren't already part of any group.</small>

- If we ran the streaming service and could "identify" the three clusters, it could help inform us on who to make recommendations to.<br><small>For example, if someone in the bottom-right cluster likes _How I Met Your Mother_, we might recommend it to other members of the bottom-right cluster since they have similar tastes.</small>

- **How do we algorithmically determine these clusters**, especially when there are too many dimensions to visualize?

### Clustering

- **Goal**: Given a set of $n$ data points stored as vectors in $\mathbb{R}^d$, $\vec x_1, \vec x_2, ..., \vec x_n$, and a positive integer $k$, **place the data points into $k$ clusters of nearby points**.<br><small>In the scatter plot below, $n = 9$ and $d = 2$.</small>

In [ ]:
util.show_ratings()

- Think of clusters as **colors**; in other words, the goal of clustering is to assign each point a color, such that points of the same color are similar to one another.

- Note, unlike with regression or classification, there is no "right answer" that we're trying to predict – there is no $y$! This is what makes clustering **unsupervised**.

### Centroids

- Idea: Points in a cluster should be close to the center of the cluster.<br><small>The clustering method we're developing relies on this assumption.</small>

- **One** technique for defining clusters involves choosing $k$ cluster centers, known as **centroids**.

    $$\vec \mu_1, \vec \mu_2, ..., \vec \mu_k \in \mathbb{R}^d$$

    For instance, $\vec \mu_2$ is the center of cluster 2.<br><small>Cluster 2 might be the set of points colored **<span style="color:blue">blue</span>**, for instance.

- These $k$ centroids define the $k$ clusters; **each data point "belongs" to the nearest centroid to it**.

- Our problem reduces to finding the **best locations** for the centroids.<br><small>Over the next few slides, we'll visualize several possible sets of centroids and the clusters they define.</small>

With the following $k = 3$ centroids, the data are colored in the way that we'd expect.

In [ ]:
util.visualize_centroids([(2, 7), (8, 4), (8, 8)])

But here, even though $k = 3$, the data are not colored "naturally"!

In [ ]:
util.visualize_centroids([(2, 7), (8, 4), (3, 7)])

Nothing is stopping us from setting $k = 2$, for instance.

In [ ]:
util.visualize_centroids([(2, 7), (8, 4)])

Or $k = 5$!

In [ ]:
util.visualize_centroids([(4, 4), (5, 5), (6, 6), (7, 7), (8, 8)])

### Reflections on choosing a centroid

- Some values of $k$ seemed more intuitive than others; $k$ is a **hyperparameter** that we'll need to tune.<br><small>More on this later.</small>

- For a fixed $k$, some clusterings "looked" better than others; we'll need a way to quantify this.

- As we did at the start of the second half of the course, we'll formulate an **objective function** to minimize. Specifically, we'll minimize **inertia**, $I$:

$$I(\vec \mu_1, \vec \mu_2, ..., \vec \mu_k) = \text{total squared distance} \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{of each point } \vec x_i \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{ to its closest centroid } \vec \mu_j$$


- Lower values of inertia lead to better clusterings; our goal is to find the set of centroids $\vec \mu_1, \vec \mu_2, ... \vec \mu_k$ that **minimize inertia**, $I$.

<div class="alert alert-success">
    
### Activity
    
Recall, inertia is defined as follows:
    
$$I(\vec \mu_1, \vec \mu_2, ..., \vec \mu_k) = \text{total squared distance} \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{of each point } \vec x_i \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{ to its closest centroid } \vec \mu_j$$
    
<br>
    
Suppose we arrange the dataset below into $k = 2$ clusters. What is the **minimum possible inertia**?
    
<center><img src="imgs/example-q.png" width=500></center>
    
</div>

## $k$-means clustering

---

### Minimizing inertia

- **Goal**: Find the centroids $\vec \mu_1, \vec \mu_2, ..., \vec \mu_k$ that minimize inertia:

$$I(\vec \mu_1, \vec \mu_2, ..., \vec \mu_k) = \text{total squared distance} \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{of each point } \vec x_i \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{ to its closest centroid } \vec \mu_j$$


- **Issue**: There is no efficient way to find the centroids that minimize inertia!

- There are $k^n$ possible assignments of points to clusters; it would be computationally infeasible to try them all.<br><small>It can be shown that finding the optimal centroid locations is NP-hard.</small>

- We can't use calculus to minimize $I$, either – we use calculus to minimize continuous functions, but the assignment of a point $\vec x_i$ to a centroid $\vec \mu_j$ is a discrete operation. 

### $k$-means clustering (i.e. Lloyd's algorithm)

- Fortunately, there's an efficient algorithm that (tries to) find the centroid locations that minimize inertia. The resulting clustering technique is called **$k$-means clustering**.<br><small>Note that this has no relation to $k$-nearest neighbors, which we used for both regression and classification. Remember that clustering is an unsupervised technique!</small>

0. **Randomly** initialize $k$ centroids.<br><small>There are other ways of initializing the centroids as well.</small>

1. Assign each point to the nearest centroid.

2. Move each centroid to the **center** of its group.<br><small>We compute the center of a group by taking the mean of the group's coordinates.</small>

3. **Repeat** steps 1 and 2 until the centroids stop changing!<br><small>This is an iterative algorithm!</small>

- Let's visualize a few iterations ourselves.

In [ ]:
util.visualize_centroids([(2, 5), (8, 10)], show_color=False, title='Step 0: Random Initialization<br>Red:(2, 5), Blue: (8, 10)')

In [ ]:
util.visualize_centroids([(2, 5), (8, 10)], title='Iteration 1, Step 1: Assign each point to the nearest centroid<br>Red:(2, 5), Blue: (8, 10)')

In [ ]:
util.visualize_centroids([(2, 5), (8, 10)], lines=True, title='Iteration 1, Step 1: Assign each point to the nearest centroid<br>Red:(2, 5), Blue: (8, 10); <b>Inertia = Sum(squared distances) = 156.25</b>')

In [ ]:
util.visualize_centroids([(3.6, 6.8), (9.5, 7.125)], lines=True, assignments=[0] * 5 + [1] * 4, title='Iteration 1, Step 2: Move each centroid to the center of its group<br>Red:(3.6, 6.8), Blue: (9.5, 7.125); <b>Inertia = 85.1875</b>')

In [ ]:
util.visualize_centroids([(3.6, 6.8), (9.5, 7.125)], assignments=[0] * 5 + [1] * 4, title='Iteration 1, Step 2: Move each centroid to the center of its group<br>Red:(3.6, 6.8), Blue: (9.5, 7.125); <b>Inertia = 85.1875</b>')

In [ ]:
util.visualize_centroids([(3.6, 6.8), (9.5, 7.125)], title='Iteration 2, Step 1: Assign each point to the nearest centroid<br>Red:(3.6, 6.8), Blue: (9.5, 7.125); <b>Inertia = 70.653125</b>')

In [ ]:
util.visualize_centroids([(2.5, 7.75), (9.2, 6.3)], title='Iteration 2, Step 2: Move each centroid to the center of its group<br>Red:(2.5, 7.75), Blue: (9.2, 6.3); <b>Inertia = 58.35</b>')

In [ ]:
util.visualize_centroids([(2.5, 7.75), (9.2, 6.3)], title='Iteration 3, Step 1: Assign each point to the nearest centroid<br>No change, so algorithm terminates!')

### Why does $k$-means work?

- On each iteration, **inertia can only stay the same or decrease** – it cannot increase.

$$I(\vec \mu_1, \vec \mu_2, ..., \vec \mu_k) = \text{total squared distance} \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{of each point } \vec x_i \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{ to its closest centroid } \vec \mu_j$$


- Why? Step 2 and step 3 **alternate** minimizing inertia in different ways:
    - In Step 2, we assign each point to the nearest centroid; this reduces the squared distance of each point to its closest centroid.
    - In Step 3, we move the centroids to the **center (mean position)** of their groups; this reduces the total **squared** distance from a centroid to the points assigned to it.

- Since there are only finitely many possible assignments of points to clusters, eventually the algorithm will terminate at some **potentially local** minimum.<br><small>Read more on the theory [**here**](https://www.cs.toronto.edu/~rgrosse/courses/csc311_f21/lectures/lec11.pdf).</small>

### Let's experiment!

- Let's visualize more runs of the algorithm [**here**](https://www.naftaliharris.com/blog/visualizing-k-means-clustering).

<center><img src="imgs/smiley.png" width=500></center>

- To replicate the picture above, select "I'll Choose" and "Smiley Face."

### In what sense is $k$-means _optimal_?

- The algorithm discussed **isn't guaranteed** to find the centroids that minimize inertia; depending on the initially-chosen centroids, it may converge at a local minimum.<br><small>One solution is $k$-means++, which picks one centroid randomly and chooses the others in a way that maximizes distance from existing centroids. Read more [here](https://en.wikipedia.org/wiki/K-means%2B%2B).</small>

- Even if $k$-means "works", the resulting clustering might not look "right" to humans. That is, the clustering that minimizes inertia doesn't necessarily look correct to us.<br><small>Remember, the core assumption in $k$-means is that **points in a cluster should be close to the center of the cluster**. This assumption isn't always true!</small>

## Choosing the number of clusters

---

### Choosing $k$ in $k$-means clustering

- Given a dataset, how do we choose $k$, the number of clusters to use?

- The larger the value of $k$, the smaller inertia is.

$$I(\vec \mu_1, \vec \mu_2, ..., \vec \mu_k) = \text{total squared distance} \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{of each point } \vec x_i \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{ to its closest centroid } \vec \mu_j$$

- If $k = n$, then each point is a centroid, and inertia is 0!

- But, the goal of clustering is to put the data into groups, so a large number of groups may not be meaningful.

In [ ]:
util.show_ratings()

### The elbow method

- For several different values of $k$, let's compute the inertia of the resulting clustering, using the scatter plot from the previous slide.

In [ ]:
util.show_elbow()

- The **elbow method** says to choose the $k$ that appears at the elbow of the plot of inertia vs. $k$, since there are diminishing returns for using more than $k$ clusters.<br><small>Above, we see an elbow at $k = 3$, which gives us the $k$ that matches our natural intuition in this example.</small>

- In practice, the data may not have natural clusters, so the choice of $k$ may not be so obvious.<br><small>And, there may be other business reasons to choose a specific value of $k$, e.g. if you're told to categorize customers of a clothing item into 5 groups: XS, small, medium, large, XL.</small>

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!

## Example: World Bank data 🌎

---

### Loading the data

- Below, we load in a dataset containing hundreds of attributes per country, taken from the [World Bank](https://databank.worldbank.org/source/world-development-indicators#).

In [ ]:
world_bank = pd.read_csv('data/world_bank_data.csv').set_index('country').fillna(0)
world_bank

- There are $d = 209$ features, far too many to visualize before clustering.

In [ ]:
world_bank.columns

- **Dimensionality reduction** is another form of unsupervised learning that would help us visualize the data; we'll explore it briefly next class.

### The elbow method, revisited

- How many clusters should we use? We'll need to resort to the elbow method, since we can't visualize the data to see how many "natural" clusters there are.

In [ ]:
util.show_elbow_world_bank(world_bank)

- The choice is a bit more ambiguous than before; here, we'll use $k = 6$.

### Clustering in `sklearn`

- To create our clusters, we'll use `KMeans` in `sklearn`.

In [ ]:
from sklearn.cluster import KMeans

- Like other models we've used in `sklearn`, we need to instantiate and fit a `KMeans` object. The difference is that the `fit` method only takes in a single `X`, not an `X` and `y`.<br><small>$k$-means is an unsupervised method, so there is no `y`.</small>

In [ ]:
# The default value of k is 8; we should generally specify another value.
# We fix a random_state for reproducibility; remember the centroids are generally initialized randomly.    
model = KMeans(n_clusters=6, random_state=15)
model.fit(world_bank)

- A fit `KMeans` instance has a `predict` method. It outputs the cluster whose centroid the data point is closest to.

In [ ]:
model.predict(world_bank.loc[['United States']])

In [ ]:
# It seems that the US and Canada are assigned to different clusters!
model.predict(world_bank.loc[['Canada']])

### Inspecting clusters

- We can view the countries assigned to each cluster.

In [ ]:
countries_and_clusters = pd.Series(model.labels_, index=world_bank.index)
util.list_countries_by_cluster(countries_and_clusters)

- It seems that the vast majority of countries are assigned to the same cluster!

### Visualizing clusters

In [ ]:
util.country_choropleth(countries_and_clusters)

### Standardize before clustering!

- Clustering, like $k$-nearest neighbors and regularization, is a **distance-based method**, meaning that it **depends on the scale of the data**.

- In `world_bank`, some features are in the millions or billions, while some are in the single digits. The larger features will influence cluster membership more than the smaller features.

In [ ]:
world_bank.iloc[[1], -9:-5]

- **Solution**: Standardize before clustering.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
model_std = make_pipeline(StandardScaler(), KMeans(n_clusters=6, random_state=15)) # We fix a random state for reproducibility.
model_std.fit(world_bank)

- Once we standarize, the sizes of the clusters seem to be a bit more evenly distributed!

In [ ]:
countries_and_clusters_std = pd.Series(model_std[-1].labels_, index=world_bank.index)
util.list_countries_by_cluster(countries_and_clusters_std)

### Visualizing clusters after standardizing

- Note that the colors themselves are arbitrary.

In [ ]:
util.country_choropleth(countries_and_clusters_std, title='after Standardizing')

## Agglomerative clustering

---

### Overview of clustering methods

- `sklearn` supports many different clustering methods! Read about them all [**here**](https://scikit-learn.org/1.5/modules/clustering.html).

<center><img src="imgs/clustering-methods.png" width=800></center>

- Remember the "no free lunch theorem" – there isn't a clustering method that is **always** better than all other clustering methods. It depends on the data.

### Agglomerative clustering

- Let's revisit the ratings dataset from earlier.

In [ ]:
util.show_ratings()

- **Agglomerative clustering**, a form of **hierarchical clustering**, creates clusters by:
    1. Starting with each point as its own cluster.
    2. Repeatedly **combining the two closest clusters** until there are only $k$ clusters remaining.

- Let's visualize it in the context of this dataset.

In [ ]:
util.color_ratings(title='Iteration 0')

- The two closest clusters are <b><span style="color:brown">cluster 7</span></b> and <b><span style="color:navy">cluster 8</span></b>, so we merge them.

In [ ]:
util.color_ratings(title='Iteration 1', labels=[0, 1, 2, 3, 4, 5, 6, 7, 7])

- Now, the two closest clusters are <b><span style="color:cyan">cluster 5</span></b> and <b><span style="color:magenta">cluster 6</span></b>, so we merge them.

In [ ]:
util.color_ratings(title='Iteration 2', labels=[0, 1, 2, 3, 4, 5, 5, 7, 7])

- It's not clear what the next merge should be – should <b><span style="color:orange">cluster 4</span></b> merge with <b><span style="color:cyan">cluster 5</span></b> or should <b><span style="color:green">cluster 2</span></b> merge with <b><span style="color:purple">cluster 3</span></b>?

### Linkage criteria

- We need a way to measure the distance between two clusters.<br><small>For example, what is the "distance" between <b><span style="color:orange">cluster 4</span></b> and <b><span style="color:cyan">cluster 5</span></b> below?</small>

In [ ]:
util.color_ratings(title='Iteration 2', labels=[0, 1, 2, 3, 4, 5, 5, 7, 7], width=500, height=400)

- The **linkage criteria** determines how to compute the distance between  two clusters.

- Some examples:
    - Average linkage: The average distance between points in both clusters.
    - Single linkage: The minimum distance between points in both clusters.
    - Complete linkage: The maximum distance between points in both clusters.

In [ ]:
util.color_ratings(title='Iteration 2', show_distances=[(1, 3), (0, 2), (0, 1), (2, 3), (4, 5)], labels=[0, 1, 2, 3, 4, 5, 5, 7, 7])

- We'll use single linkage, i.e.:

$$\text{distance(cluster $A$, cluster $B$)} = \text{minimum distance between} \\ \:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\:\: \text{any point in $A$ and any point in $B$}$$

- Here, there are lots of ties; we'll arbitrarily choose to merge <b><span style="color:orange">cluster 4</span></b> and <b><span style="color:cyan">cluster 5</span></b>.

In [ ]:
util.color_ratings(title='Iteration 3', show_distances=[(0, 2), (2, 3)], labels=[0, 1, 2, 3, 5, 5, 5, 7, 7])

- Again, there's a tie; we'll arbitrarily choose to merge <b><span style="color:green">cluster 2</span></b> and <b><span style="color:purple">cluster 3</span></b>.<br><small>We could have also merged <b><span style="color:green">cluster 2</span></b> and <b><span style="color:red">cluster 0</span></b>, since their minimum distance is also the same.</small>

In [ ]:
util.color_ratings(title='Iteration 4', show_distances=[(0, 2), (1, 2)], labels=[0, 1, 2, 2, 5, 5, 5, 7, 7])

- Next, we merge <b><span style="color:green">cluster 2</span></b> and <b><span style="color:red">cluster 0</span></b>.<br><small>Why? Because the minimum distance between <b><span style="color:red">cluster 0</span></b> and <b><span style="color:green">cluster 2</span></b> is less than the minimum distance between <b><span style="color:blue">cluster 1</span></b> and <b><span style="color:green">cluster 2</span></b>.

In [ ]:
util.color_ratings(title='Iteration 5', labels=[2, 1, 2, 2, 5, 5, 5, 7, 7])

- And finally, we merge <b><span style="color:green">cluster 2</span></b> and <b><span style="color:blue">cluster 1</span></b>.

- If we just want $k = 3$ clusters, we stop here! If we wanted $k = 2$ clusters, we'd then merge the two closest clusters, based on the single linkage criterion.

### $k$-means vs. agglomerative clustering

- On what sorts of datasets does agglomerative clustering perform better than $k$-means clustering?

In [ ]:
util.show_scatter_comp()

In [ ]:
util.show_scatter_comp_k_means(k=2)

- Note that $k$-means clustering optimizes for inertia, not "blobiness."<br>It doesn't work well when the natural clusters are of uneven sizes.<br><small>Read more [**here**](https://stats.stackexchange.com/questions/133656/how-to-understand-the-drawbacks-of-k-means).</small>

In [ ]:
util.show_scatter_comp_agg(k=2)

- Another metric that's used to compare different clusterings is the **silhouette score**.<br><small>Read more [**here**](https://en.wikipedia.org/wiki/Silhouette_(clustering)).</small>

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
Remember that you can always ask questions anonymously at the link above!